## 6. Classification
### 1. Merge one hot encoded genres to lyrics

In [1]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
df_val = pd.read_csv('../dataset/Lyrics_en_artists_clean_test.csv', index_col=['Band'])
df

,Lyrics,acid,acoustic,adult,age,album,alternative,americana,and,anthem,...,0,1,2,3,4,5,6,7,8,9
Band,,,,,,,,,,,,,,,,,,,,,
Leo Sayer,Farewell old friend\r\r\r\r\r\r\nLeave me now ...,0,0,1,0,0,0,0,0,0,...,-0.137803,1.807695,0.261667,0.321838,0.058538,-0.939253,-0.300022,0.014185,0.000127,-0.435531
Rick Springfield,Now I can tell you how I feel\r\r\r\r\r\r\nA b...,0,0,0,0,1,0,0,0,0,...,-0.995474,1.659972,0.087044,0.009426,0.593562,-0.604191,-0.211119,-0.275529,-0.885951,0.145885
Katie Melua,You're like a secret symphony\r\r\r\r\r\r\nTha...,0,0,0,0,0,0,0,0,0,...,0.611690,-0.307951,-0.091200,-0.353698,0.087799,0.157082,-0.080834,-0.420153,0.030168,0.150349
Ike Quebec,I could show the world how to smile\r\r\r\r\r\...,0,0,0,0,0,0,0,0,0,...,0.643051,-0.590612,-0.206580,-0.768192,1.540291,0.152065,0.522778,-0.455186,0.208514,-1.005969
Tori Amos,"Load up on guns, bring your friends\r\r\r\r\r\...",0,0,0,0,0,0,0,0,0,...,-1.008280,1.238291,-0.057360,0.317390,0.365412,0.628785,-0.261243,-0.630747,0.113289,0.342137
Marvin Gaye,Baby please stay\r\r\r\r\r\r\nDon't go away ba...,0,0,0,0,0,0,0,0,0,...,0.055658,-0.387400,-0.181425,-0.355558,0.970810,-0.598812,-0.392279,1.139917,0.380985,0.640052
Sly & Robbie,That may be all I need\r\r\r\r\r\r\nIn darknes...,0,0,0,0,0,0,0,0,0,...,-0.070802,-0.372412,0.041041,-0.238111,-0.152987,-0.107222,-0.386034,-0.190035,0.073516,0.155998
Keb' Mo',"Ain't got no money, nowhere to sleep\r\r\r\r\r...",0,0,0,0,0,0,0,0,0,...,-0.146235,-0.474542,0.769208,0.103429,0.018419,-0.019159,-0.075911,-0.063504,0.525616,0.293656
The Searchers,Sugar and spice and all things nice\r\r\r\r\r\...,0,0,1,0,0,0,0,0,0,...,0.455524,1.593258,0.104360,0.648141,-0.517314,-0.458454,-0.292488,0.477262,0.539191,-1.005526


### 2. Label exploration  

In [2]:
%matplotlib inline

y_train = df.drop(df.columns[:-10], axis=1)
y_test = df_val.drop(df.columns[:-10], axis=1)
test_lyrics = df_val['Lyrics']
del df
del df_val
#pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

### 3. Doc2Vec Deep learning LSTM classifier

In [3]:
from gensim.models.doc2vec import Doc2Vec

doc2vec = Doc2Vec.load("../dataset/doc2vec")
doc_vectors = doc2vec.wv
print("Number of word vectors: {}".format(len(doc_vectors.vocab)))

C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Number of word vectors: 77854


In [4]:
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['s', 'chorus'])

# python -m spacy download en
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    
def transform_lyric(lyric):
    lyric = simple_preprocess(str(lyric), deacc=True)

    new_lyric = list()
    for word in lyric:
        if(word not in stop_words):
            new_lyric.append(word)
    lyric = new_lyric


    lemma_lyric = list()
    doc = nlp(" ".join(lyric)) 
    lemma_lyric = list()
    for token in doc:
        if(token.pos_ in allowed_postags):
            lemma_lyric.append(token.lemma_)
    lyric = lemma_lyric
    return lyric

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import numpy as np
Text_INPUT_DIM = 100
train_size = y_train.shape[0] 
test_size = y_test.shape[0] 
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))
text_test_arrays = np.zeros((test_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = doc2vec.docvecs[i]
    
for i in range(test_size):
    test_lyrics[i] = transform_lyric(test_lyrics[i])
    text_test_arrays[i] = doc2vec.infer_vector(test_lyrics[i])
#text_test_arrays = doc2vec.infer_vector(test_lyrics)
#del test_lyrics

In [6]:
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Dense(256, input_dim=Text_INPUT_DIM, init='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, init='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, init='normal'))

model.compile(loss='mse', optimizer='adam',    metrics=['accuracy'])
model.summary()

estimator = model.fit(text_train_arrays, y_train, validation_data=(text_test_arrays,y_test), epochs = 10, batch_size=1024)

C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=100, activation="relu", kernel_initializer="normal")`
  
C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_initializer="normal")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="normal")`
  if sys.path[0] == '':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 256)               25856     
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                2570      
Total params: 94,218
Trainable params: 94,218
Non-trainable params: 0
_________________________________________________________________
Train on 433385 samples, validate on 108347 samples
Epoch 1/10
433385/433385 [==============================] - 4s 8us/step - loss: 0.2237

In [29]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
train_lyrics = df['Lyrics']
del df

In [30]:
#Bryce Fox
#Lucy
#genres: "indie poptimism","modern alternative rock","modern rock"

lyric = "So bring your vibe over here here here \
Throw it up in the air air air, oh \
I run my hands through your hair hair hair \
And give you that love cuz you're oh so rare \
You look better over here then take you do over there \
And I don't wanna share \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I'll fix you up something real real tall \
It's looking up, we should take it all off \
Yeah \
You remind me why the stars don't fall \
I swear I'll love you like your last name's Ball \
I'll take you to the lair if we make it past the stairs \
I don't wanna share, yeah \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
 \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
Yeah \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
(So bite your tongue before I bite it for ya) \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
She said she ain't Lucy \
I said not yet" 

lyric = transform_lyric(lyric)
print(lyric)

['bring', 'vibe', 'throw', 'air', 'air', 'air', 'run', 'hand', 'hair', 'hair', 'hair', 'give', 'love', 'rare', 'look', 'better', 'take', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'fix', 'something', 'real', 'real', 'tall', 'look', 'take', 'remind', 'star', 'fall', 'swear', 'love', 'last', 'name', 'ball', 'take', 'lair', 'make', 'past', 'stair', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'say', 'tryna', 'get', 'know', 'bite', 'tongue', 'bite', 'say', 'tryna', 'get', 'know

In [31]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

[Talking]
Yeah, pimps up hoes down, yeah 
Hustlas, playas, gangstas, gangstas, yeah
Fa shiggedel, shiggedy, uh, click shit down
What is it, check this out

[E-40]
Raised in the heart of the ghetto, dipping and dodging the metro
Pitching the ? and coke that's what I use to pedal
Heavy metal, a j-jack of all trays
Then you pimp, that's a hustler in many ways

Acting bad in the traffic, the hustler with the package
Serving that cha-cha, that yell, they all tragic magic
Plastic baggage, jelly jars in microwaves
Got to have it, backwoods and purple hayes

Mess around in my side of town, get clowned down
John, Jane Doe, lost and found
Everybody know this young player's about his business
Riches, chickens I pimp, but lickeness

My L.I.P.I folks be digging this
Devon, Pimpy Gear, Max Queezthis
Gorilla, Scarp Down, no conscience
In this occupation you can't be generous

Me and my fellas be bossing, dipping the pander and flossing
No matter how much it's costing we do this often cause we

[Choru

In [32]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.28408453,
 'pop': 0.2541779,
 'jazz': 0.16478156,
 'christmas': 0.12639225,
 'country': 0.11700781,
 'standards': 0.114419326,
 'adult': 0.114407666,
 'alternative': 0.10623583,
 'folk': 0.10595578,
 'vocal': 0.09854613}

In [33]:
lyric = "I hear you talkin' shit, bro you think you're the heat \
Please bow down to defeat you're barely mince meat \
Stop with the street talk, and start to do the street, walk \
Lock yourself in and tell me this, how you gonna battle with this sick shit that I spit bitch \
See you still flappin' your jaws, prepare to be thrown in the ocean \
In the middle with jaws \
Or prefer to be served and severed with claws, knock on the doors Of absolute death you may \
But please be ready for the automatic failures you may make \
So take this and wait for another dismembering, remembering \
The tethering of your dream and how it was minced to pieces \
Believe in yourself please, to save you now you'll be needin' \
Help from Jesus \
You think I'm copy, pastin' please, tell me how you're actin' up \
Better be quite or Imma bout to bitch smack your mouth shut \
For good, yeah man talk the hood talk better go back and walk the hood walk you ain't had the taste of a real G \
Get back down to your level and prepare to be beat \
Writin' on your sheet, like you're deep please bitch your weak \
You ain't on my level or a level at all, take a vacation \
Up the damn wall and fall \
Humpty Dumpty, I must be rusty but you better knock the shit Down if you goin' against me \
I'm hard to beat, you're hardly cheese compared to these beats And this emcee \
You ready, alright, so come on down, I bet you to do it \
Let's see if you got the balls or if you're just truant \
To me, yeah let's keep an eye, on your ass as the heat intensifies \
Talk to me in a smart tone, prepare to meet your demise \
Throw you, a hundred million miles is what im finna do \
Call me Son Goku \
I already told you, what I'm gonna do, finna plow through \
Yeah man try to tackle a topic more intense then mine \
I always rap from the heart, make the fake rappers sigh \
But I really really don't give at all a flying fuck \
My chance of making this shit is the same as seein' a flyin' truck \
Or it could all depend on some serious luck \
Maybe I'll release a track that is heard my millions \
Make it big and to the top and make billions \
But that's a dream for another day, I'm here to take you down \
Don't care what you say \
Freestylin' this rap shit to the grave \
Rap battlin' bitches in my spare time for loose change \
Please, all of you fake ass rappers are plan \
Make like like a plane and take the next trip away \
Far away, from here I don't need to here your shit anymore \
I'm tired of it now, next place to be, will be a fuckin' morgue \
But let me just let you all know this, I'm sick of it, all of it \
So listen next time I write another track \
Listen to the facts I spit, and find it through my old wack rap crap \
I know I've said some shit, that know I demise \
But just look at my own and, your own life \
But this time, through my eyes \
I got a couple little bitches that need a little call out \
But I'll reside, got more and more, don't get me wrong \
Now get the fuck back home and call your mom \
Continue to rap please, just leave me alone \
I'm done hearing wack disses like this \
I'm sick of this, listening to this petty as shit \
Bitch"
lyric = transform_lyric(lyric)
print(lyric)

['hear', 'talkin', 'shit', 'bro', 'think', 'heat', 'bow', 'defeat', 'barely', 'mince', 'meat', 'stop', 'street', 'talk', 'start', 'street', 'walk', 'lock', 'tell', 'go', 'battle', 'sick', 'shit', 'spit', 'bitch', 'see', 'still', 'flappin', 'jaw', 'prepare', 'throw', 'ocean', 'middle', 'jaw', 'prefer', 'serve', 'sever', 'claws', 'knock', 'door', 'absolute', 'death', 'may', 'please', 'ready', 'automatic', 'failure', 'may', 'make', 'take', 'wait', 'dismembering', 'remember', 'tethering', 'dream', 'mince', 'piece', 'believe', 'save', 'needin', 'help', 'think', 'copy', 'pastin', 'tell', 'actin', 'better', 'quite', 'imma', 'bout', 'bitch', 'smack', 'mouth', 'shut', 'good', 'talk', 'hood', 'talk', 'better', 'go', 'back', 'walk', 'hood', 'walk', 'taste', 'real', 'get', 'back', 'level', 'prepare', 'beat', 'writin', 'sheet', 'deep', 'bitch', 'weak', 'level', 'level', 'take', 'vacation', 'damn', 'wall', 'fall', 'humpty', 'dumpty', 'must', 'rusty', 'better', 'knock', 'shit', 'go', 'hard', 'beat', 

In [34]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

Girl why do you got that look on your face 
Time is now girl I don't want to waste 
You shocked my eye when you stepped in the place 
I want to get with that baby 
Now there's no way I can let
Let you get past me now baby 
I've scoped you girl now you're mine 
It's time to let  
Let you shine all night baby 
Love girl let me take you out 
Show you around my town 
Look at them starin' at you baby 
Tonight girl you deserve some wine 
Here's to you being so fly 
Let me just let you shine now lady 
Girl why do you got an empty glass in your hand 
You got your girls but do you have a man? 
I feel your vibe you look so good 
I can't stand to look your way baby 
Tip the cup back long legs bare breasted 
The type of body that got my mind manifested 
Lyrical potion took me at the glance when 
I see you dance 
Highlight from the light from the back 
I like the way you make me feel 
Got that sex appeal 
So swing it over this way tonight 
I want to make a long stay 0.6061241626739502 23851
One mor

In [35]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.28408453,
 'pop': 0.2541779,
 'jazz': 0.16478156,
 'christmas': 0.12639225,
 'country': 0.11700781,
 'standards': 0.114419326,
 'adult': 0.114407666,
 'alternative': 0.10623583,
 'folk': 0.10595578,
 'vocal': 0.09854613}